# General Workflow for Generating EES in 2020 Science Advances Manuscript
All scripts were executed in Python 3.7.3. We used MELD v0.0, MAGIC, v1.5.5, and SCPREP v0.12 packages from the Krishnaswami lab, using their examples as a foundation. Please reach out to Corey Duke (cgduke@uab.edu) or Jeremy Day (jjday@uab.edu) if you have any questions about our specific workflow.

In [1]:
# import packages and dependencies
import pandas as pd
import numpy as np
import graphtools as gt
import phate
import magic
import scprep
import meld

In [2]:
# setting defaults for matplotlib font sizes
import matplotlib.pyplot as plt
plt.rc('font', size=12)

In [3]:
# making sure plots & clusters are reproducible
np.random.seed(42)

In [4]:
# loading in csv file containing data. This was generated from the seurat object All_Groups_log assay of read counts.
data = scprep.io.load_csv('Data/culture/All_Counts_Culture.csv', cell_axis='column')


In [5]:
#loading in csv file containing metadata. This was generated from All_Groups_log@meta.data in seurat.
metadata = scprep.io.load_csv('Data/culture/All_Counts_Metadata_Culture.csv')
 

In [6]:
# map cluster names to cluster ids
def cluster (row):
   if row['seurat_clusters'] == 0 :
      return 'GABA-1'
   if row['seurat_clusters'] == 1 :
      return 'Drd1'
   if row['seurat_clusters'] == 2 :
      return 'GABA-2'
   if row['seurat_clusters'] == 3 :
      return 'Poly'  
   if row['seurat_clusters'] == 4 :
      return 'Drd2'
metadata['cluster'] = metadata.apply (lambda row: cluster(row), axis=1)

# Dopamine Stim

In [7]:
#cluster metadata and data by treatment
is_tx = metadata['Stim'] == 'Veh'
x = metadata[is_tx]
is_tx2 = metadata['Stim'] == 'DA'
y = metadata[is_tx2]
metadata_da = pd.concat([x,y])

#cluster data by treatment
indexNames = metadata_da.index
ids=data.index.isin(metadata_da.index) == True
data_da = scprep.select.select_rows(data, idx=ids)

In [8]:
# Create an indicator array for the sample. This is also the Raw Experimental Signal (RES).
metadata_da['RES'] = np.array([-1 if label.startswith('Veh') else 1 for label in metadata_da['Stim']])


In [9]:
# sort clusters by abundance
cluster_abundance = metadata_da.groupby('cluster').aggregate({'RES':np.mean}).sort_values('RES')
cluster_abundance['newClusterID'] = np.arange(cluster_abundance.shape[0])


In [10]:
# relabel existing clusters in metadata
new_clusters = cluster_abundance.loc[metadata_da['cluster']]['newClusterID']
new_clusters.index = metadata_da.index
metadata_da['clusterID'] = new_clusters

In [11]:
# remove lowly expressed genes
data_da = scprep.filter.filter_rare_genes(data_da, min_cells=5)

In [12]:
# create the primary graph
G = gt.Graph(data_da, knn=9, decay=10, n_pca=100, use_pygsp=True, n_jobs=-2, verbose=True)

Calculating PCA...
Calculated PCA in 1.09 seconds.
Calculating KNN search...
Calculated KNN search in 0.12 seconds.
Calculating affinities...
Calculated affinities in 0.20 seconds.


In [13]:
#impute gene expression using MAGIC to calculate EES
magic_op = magic.MAGIC(knn=G.knn, decay=G.decay)
data_da_magic = magic_op.fit_transform(data_da, graph=G)

Calculating MAGIC...
  Running MAGIC on 1449 cells and 13646 genes.
  Using precomputed graph and diffusion operator...
  Calculating imputation...
    Automatically selected t = 14
  Calculated imputation in 0.35 seconds.
Calculated MAGIC in 0.72 seconds.


In [14]:
# generate EES using the main graph and magic transformed data
meld_op = meld.MELD()
cocaine_ees = meld_op.fit_transform(metadata_da['RES'], G)
# mean center the EES
cocaine_ees = cocaine_ees - np.mean(cocaine_ees)

metadata_da['EES'] = cocaine_ees

In [15]:
#Build EES gene correlation dataframe and tables. Calculate and include Log2FC in tables.
#Note: Will give an expected divide by zero error, as we are not adding an integer to get around the log2 fold change
# divide by zero issue, which would artificially penalize lowly expressed genes.

import warnings; #these two lines will disable display of the divide by zero warning mentioned above.
warnings.filterwarnings('ignore');

cluster_list = ['Drd1','Drd2','GABA-1','GABA-2','Poly']

for j in cluster_list:
    cluster_name = j

    # calc logfc - un-normalize data (nonlogp1)
    is_in_cluster = metadata_da['cluster'] == cluster_name
    data_cluster, metadata_cluster = scprep.select.select_rows(data_da, metadata_da, idx=is_in_cluster)
    data_cluster = (np.e ** data_cluster) -1
    tx_name = 'Veh'
    is_in_tx = metadata_cluster['Stim'] == tx_name
    data_tx, metadata_tx = scprep.select.select_rows(data_cluster, metadata_cluster, idx=is_in_tx)
    tx_name = 'DA'
    is_in_tx = metadata_cluster['Stim'] == tx_name
    data_tx2, metadata_tx2 = scprep.select.select_rows(data_cluster, metadata_cluster, idx=is_in_tx)

    # calc fold change (non log plus one)
    mean_veh = data_tx.mean(axis=0)
    mean_tx = data_tx2.mean(axis=0)
    FC = mean_tx / mean_veh
    FC_log = np.log2(FC)
    FC_log = FC_log.replace([np.inf, -np.inf], np.nan)
    FC_log = FC_log.dropna()
    fc_genes = FC_log.reset_index()
    fc_genes.columns = ['gene','logFC']

    # calculate pearson's r
    is_in_cluster = metadata_da['cluster'] == cluster_name
    data_cluster, metadata_cluster = scprep.select.select_rows(data_da, metadata_da, idx=is_in_cluster)

    gene_list = fc_genes['gene']
    gene_list = np.array(gene_list)

    fc_list = fc_genes['logFC']
    fc_list = np.array(fc_list)

    dremi_list = []
    for i in gene_list:
        value = scprep.stats.pairwise_correlation(metadata_cluster['EES'], data_cluster[i])
        dremi_list.append(value)

    # rank by pearson and build dataframe
    dremi_cluster = pd.DataFrame({'Gene':gene_list, 'r_val':dremi_list, 'log2FC':fc_list})
    dremi_cluster = dremi_cluster.sort_values(by=['r_val'], ascending=False)
    dremi_cluster['r_Rank'] = [1 * i for i in range(len(dremi_cluster))]
    dremi_cluster['r_val'] = dremi_cluster['r_val'].str[0] #remove brackets
    dremi_cluster['r_val'] = dremi_cluster['r_val'].str[0]
    
    #save cluster data
    dremi_cluster.to_csv('Data/DA_'+cluster_name+'_Pearson_FC_by_Gene(no_cutoff).csv', index=False, header=True)



# SKF Stim

In [16]:
#cluster metadata and data by treatment
is_tx = metadata['Stim'] == 'Veh'
x = metadata[is_tx]
is_tx2 = metadata['Stim'] == 'SKF'
y = metadata[is_tx2]
metadata_skf = pd.concat([x,y])

#cluster data by treatment
indexNames = metadata_skf.index
ids=data.index.isin(metadata_skf.index) == True
data_skf = scprep.select.select_rows(data, idx=ids)

In [17]:
# Create an indicator array for the sample. This is also the Raw Experimental Signal (RES).
metadata_skf['RES'] = np.array([-1 if label.startswith('Veh') else 1 for label in metadata_skf['Stim']])


In [18]:
# sort clusters by abundance
cluster_abundance = metadata_skf.groupby('cluster').aggregate({'RES':np.mean}).sort_values('RES')
cluster_abundance['newClusterID'] = np.arange(cluster_abundance.shape[0])


In [19]:
# relabel existing clusters in metadata
new_clusters = cluster_abundance.loc[metadata_skf['cluster']]['newClusterID']
new_clusters.index = metadata_skf.index
metadata_skf['clusterID'] = new_clusters

In [20]:
# remove lowly expressed genes
data_skf = scprep.filter.filter_rare_genes(data_skf, min_cells=5)

In [21]:
# create the primary graph
G = gt.Graph(data_skf, knn=9, decay=10, n_pca=100, use_pygsp=True, n_jobs=-2, verbose=True)

Calculating PCA...
Calculated PCA in 1.18 seconds.
Calculating KNN search...
Calculated KNN search in 0.11 seconds.
Calculating affinities...
Calculated affinities in 0.23 seconds.


In [22]:
#impute gene expression using MAGIC to calculate EES
magic_op = magic.MAGIC(knn=G.knn, decay=G.decay)
data_skf_magic = magic_op.fit_transform(data_skf, graph=G)

Calculating MAGIC...
  Running MAGIC on 1599 cells and 13784 genes.
  Using precomputed graph and diffusion operator...
  Calculating imputation...
    Automatically selected t = 15
  Calculated imputation in 0.41 seconds.
Calculated MAGIC in 0.79 seconds.


In [23]:
# generate EES using the main graph and magic transformed data
meld_op = meld.MELD()
cocaine_ees = meld_op.fit_transform(metadata_skf['RES'], G)
# mean center the EES
cocaine_ees = cocaine_ees - np.mean(cocaine_ees)

metadata_skf['EES'] = cocaine_ees

In [24]:
#Build EES gene correlation dataframe and tables. Calculate and include Log2FC in tables.
#Note: Will give an expected divide by zero error, as we are not adding an integer to get around the log2 fold change
# divide by zero issue, which would artificially penalize lowly expressed genes.

import warnings; #these two lines will disable display of the divide by zero warning mentioned above.
warnings.filterwarnings('ignore');

cluster_list = ['Drd1','Drd2','GABA-1','GABA-2','Poly']

for j in cluster_list:
    cluster_name = j

    # calc logfc - un-normalize data (nonlogp1)
    is_in_cluster = metadata_skf['cluster'] == cluster_name
    data_cluster, metadata_cluster = scprep.select.select_rows(data_skf, metadata_skf, idx=is_in_cluster)
    data_cluster = (np.e ** data_cluster) -1
    tx_name = 'Veh'
    is_in_tx = metadata_cluster['Stim'] == tx_name
    data_tx, metadata_tx = scprep.select.select_rows(data_cluster, metadata_cluster, idx=is_in_tx)
    tx_name = 'SKF'
    is_in_tx = metadata_cluster['Stim'] == tx_name
    data_tx2, metadata_tx2 = scprep.select.select_rows(data_cluster, metadata_cluster, idx=is_in_tx)

    # calc fold change (non log plus one)
    mean_veh = data_tx.mean(axis=0)
    mean_tx = data_tx2.mean(axis=0)
    FC = mean_tx / mean_veh
    FC_log = np.log2(FC)
    FC_log = FC_log.replace([np.inf, -np.inf], np.nan)
    FC_log = FC_log.dropna()
    fc_genes = FC_log.reset_index()
    fc_genes.columns = ['gene','logFC']

    # calculate pearson's r
    is_in_cluster = metadata_skf['cluster'] == cluster_name
    data_cluster, metadata_cluster = scprep.select.select_rows(data_skf, metadata_skf, idx=is_in_cluster)

    gene_list = fc_genes['gene']
    gene_list = np.array(gene_list)

    fc_list = fc_genes['logFC']
    fc_list = np.array(fc_list)

    dremi_list = []
    for i in gene_list:
        value = scprep.stats.pairwise_correlation(metadata_cluster['EES'], data_cluster[i])
        dremi_list.append(value)

    # rank by pearson and build dataframe
    dremi_cluster = pd.DataFrame({'Gene':gene_list, 'r_val':dremi_list, 'log2FC':fc_list})
    dremi_cluster = dremi_cluster.sort_values(by=['r_val'], ascending=False)
    dremi_cluster['r_Rank'] = [1 * i for i in range(len(dremi_cluster))]
    dremi_cluster['r_val'] = dremi_cluster['r_val'].str[0] #remove brackets
    dremi_cluster['r_val'] = dremi_cluster['r_val'].str[0]
    
    #save cluster data
    dremi_cluster.to_csv('Data/SKF_'+cluster_name+'_Pearson_FC_by_Gene(no_cutoff).csv', index=False, header=True)



# KCl Stim

In [25]:
#cluster metadata and data by treatment
is_tx = metadata['Stim'] == 'Veh'
x = metadata[is_tx]
is_tx2 = metadata['Stim'] == 'KCl'
y = metadata[is_tx2]
metadata_kcl = pd.concat([x,y])

#cluster data by treatment
indexNames = metadata_kcl.index
ids=data.index.isin(metadata_kcl.index) == True
data_kcl = scprep.select.select_rows(data, idx=ids)

In [26]:
# Create an indicator array for the sample. This is also the Raw Experimental Signal (RES).
metadata_kcl['RES'] = np.array([-1 if label.startswith('Veh') else 1 for label in metadata_kcl['Stim']])


In [27]:
# sort clusters by abundance
cluster_abundance = metadata_kcl.groupby('cluster').aggregate({'RES':np.mean}).sort_values('RES')
cluster_abundance['newClusterID'] = np.arange(cluster_abundance.shape[0])


In [28]:
# relabel existing clusters in metadata
new_clusters = cluster_abundance.loc[metadata_kcl['cluster']]['newClusterID']
new_clusters.index = metadata_kcl.index
metadata_kcl['clusterID'] = new_clusters

In [29]:
# remove lowly expressed genes
data_kcl = scprep.filter.filter_rare_genes(data_kcl, min_cells=5)

In [30]:
# create the primary graph
G = gt.Graph(data_kcl, knn=9, decay=10, n_pca=100, use_pygsp=True, n_jobs=-2, verbose=True)

Calculating PCA...
Calculated PCA in 1.43 seconds.
Calculating KNN search...
Calculated KNN search in 0.12 seconds.
Calculating affinities...
Calculated affinities in 0.22 seconds.


In [31]:
#impute gene expression using MAGIC to calculate EES
magic_op = magic.MAGIC(knn=G.knn, decay=G.decay)
data_kcl_magic = magic_op.fit_transform(data_kcl, graph=G)

Calculating MAGIC...
  Running MAGIC on 1769 cells and 14012 genes.
  Using precomputed graph and diffusion operator...
  Calculating imputation...
    Automatically selected t = 14
  Calculated imputation in 0.37 seconds.
Calculated MAGIC in 0.85 seconds.


In [32]:
# generate EES using the main graph and magic transformed data
meld_op = meld.MELD()
cocaine_ees = meld_op.fit_transform(metadata_kcl['RES'], G)
# mean center the EES
cocaine_ees = cocaine_ees - np.mean(cocaine_ees)

metadata_kcl['EES'] = cocaine_ees

In [33]:
#Build EES gene correlation dataframe and tables. Calculate and include Log2FC in tables.
#Note: Will give an expected divide by zero error, as we are not adding an integer to get around the log2 fold change
# divide by zero issue, which would artificially penalize lowly expressed genes.

import warnings; #these two lines will disable display of the divide by zero warning mentioned above.
warnings.filterwarnings('ignore');

cluster_list = ['Drd1','Drd2','GABA-1','GABA-2','Poly']

for j in cluster_list:
    cluster_name = j

    # calc logfc - un-normalize data (nonlogp1)
    is_in_cluster = metadata_kcl['cluster'] == cluster_name
    data_cluster, metadata_cluster = scprep.select.select_rows(data_kcl, metadata_kcl, idx=is_in_cluster)
    data_cluster = (np.e ** data_cluster) -1
    tx_name = 'Veh'
    is_in_tx = metadata_cluster['Stim'] == tx_name
    data_tx, metadata_tx = scprep.select.select_rows(data_cluster, metadata_cluster, idx=is_in_tx)
    tx_name = 'KCl'
    is_in_tx = metadata_cluster['Stim'] == tx_name
    data_tx2, metadata_tx2 = scprep.select.select_rows(data_cluster, metadata_cluster, idx=is_in_tx)

    # calc fold change (non log plus one)
    mean_veh = data_tx.mean(axis=0)
    mean_tx = data_tx2.mean(axis=0)
    FC = mean_tx / mean_veh
    FC_log = np.log2(FC)
    FC_log = FC_log.replace([np.inf, -np.inf], np.nan)
    FC_log = FC_log.dropna()
    fc_genes = FC_log.reset_index()
    fc_genes.columns = ['gene','logFC']

    # calculate pearson's r
    is_in_cluster = metadata_kcl['cluster'] == cluster_name
    data_cluster, metadata_cluster = scprep.select.select_rows(data_kcl, metadata_kcl, idx=is_in_cluster)

    gene_list = fc_genes['gene']
    gene_list = np.array(gene_list)

    fc_list = fc_genes['logFC']
    fc_list = np.array(fc_list)

    dremi_list = []
    for i in gene_list:
        value = scprep.stats.pairwise_correlation(metadata_cluster['EES'], data_cluster[i])
        dremi_list.append(value)

    # rank by pearson and build dataframe
    dremi_cluster = pd.DataFrame({'Gene':gene_list, 'r_val':dremi_list, 'log2FC':fc_list})
    dremi_cluster = dremi_cluster.sort_values(by=['r_val'], ascending=False)
    dremi_cluster['r_Rank'] = [1 * i for i in range(len(dremi_cluster))]
    dremi_cluster['r_val'] = dremi_cluster['r_val'].str[0] #remove brackets
    dremi_cluster['r_val'] = dremi_cluster['r_val'].str[0]
    
    #save cluster data
    dremi_cluster.to_csv('Data/KCl_'+cluster_name+'_Pearson_FC_by_Gene(no_cutoff).csv', index=False, header=True)

